In [ ]:
# | default_exp _helpers.api_docs_helper

In [ ]:
# | export

from typing import *
import types
import enum
import re
from inspect import Signature, signature, isfunction, isclass, getmembers, getdoc

from griffe.dataclasses import Docstring, Function, Parameters, Parameter, ParameterKind
from griffe.docstrings.parsers import Parser, parse
from griffe.expressions import Expression, Name

from mkdocstrings_handlers.python.handler import get_handler, PythonHandler
from markdown.core import Markdown

In [ ]:
# | export


def _convert_union_to_optional(annotation_str: str) -> str:
    """Convert the 'Union[Type1, Type2, ..., NoneType]' to 'Optional[Type1, Type2, ...]' in the given annotation string

    Args:
        annotation_str: The type annotation string to convert.

    Returns:
        The converted type annotation string.
    """
    pattern = r"Union\[(.*)?,\s*NoneType\s*\]"
    match = re.search(pattern, annotation_str)
    if match:
        union_type = match.group(1)
        optional_type = f"Optional[{union_type}]"
        return re.sub(pattern, optional_type, annotation_str)
    else:
        return annotation_str

In [ ]:
fixtures = [
    {
        "input": "arg_1: Union[int, NoneType] = 80",
        "expected": "arg_1: Optional[int] = 80",
    },
    {
        "input": "arg_1: Union[Dict[str, str], NoneType]",
        "expected": "arg_1: Optional[Dict[str, str]]",
    },
    {
        "input": "arg_1: Union[Dict[str, str], str]",
        "expected": "arg_1: Union[Dict[str, str], str]",
    },
    {
        "input": "arg_1: str",
        "expected": "arg_1: str",
    },
    {
        "input": "arg_1: bool = False",
        "expected": "arg_1: bool = False",
    },
    {
        "input": "prefix: str = 'to_'",
        "expected": "prefix: str = 'to_'",
    },
]

for fixture in fixtures:
    actual = _convert_union_to_optional(fixture["input"])
    print(actual)
    assert actual == fixture["expected"]

arg_1: Optional[int] = 80
arg_1: Optional[Dict[str, str]]
arg_1: Union[Dict[str, str], str]
arg_1: str
arg_1: bool = False
prefix: str = 'to_'


In [ ]:
# | export


def _get_arg_list_with_signature(_signature: Signature, return_as_list: bool = False) -> Union[str, List[str]]:
    """Converts a function's signature into a string representation of its argument list.

    Args:
        _signature (signature): The signature object for the function to convert.

    Returns:
        str: A string representation of the function's argument list.
    """
    arg_list = []
    for param in _signature.parameters.values():
        arg_list.append(_convert_union_to_optional(str(param)))

    return arg_list if return_as_list else ", ".join(arg_list)

In [ ]:
def fixture_function(
    arg_1: str, arg_2, arg_3: Union[Dict[str, str], str], arg_4: Optional[int] = 80
) -> str:
    pass


_signature = signature(fixture_function)

expected = (
    "arg_1: str, arg_2, arg_3: Union[Dict[str, str], str], arg_4: Optional[int] = 80"
)
actual = _get_arg_list_with_signature(_signature)

print(actual)
assert actual == expected

arg_1: str, arg_2, arg_3: Union[Dict[str, str], str], arg_4: Optional[int] = 80


In [ ]:
# | export

def _get_return_annotation(sig: Signature, symbol_definition: str) -> str:
    if sig.return_annotation and "inspect._empty" not in str(
        sig.return_annotation
    ):
        if isinstance(sig.return_annotation, type):
            symbol_definition = symbol_definition + f" -> {sig.return_annotation.__name__}\n"
        else:
            symbol_definition = symbol_definition + f" -> {sig.return_annotation}\n"
            symbol_definition = symbol_definition.replace("typing.", "")

    else:
        symbol_definition = symbol_definition + " -> None\n"
    return symbol_definition

In [ ]:
def fixture_function() -> str:
    pass

sig = signature(fixture_function)
symbol_definition = ""
actual = _get_return_annotation(sig, symbol_definition)
expected = " -> str\n"
print(actual)

assert actual == expected

 -> str



In [ ]:
# | export


def _get_symbol_definition(symbol: Union[types.FunctionType, Type[Any]]) -> str:
    """Return the definition of a given symbol.

    Args:
        symbol: A function or method object to get the definition for.

    Returns:
        A string representing the function definition
    """
    _signature = signature(symbol)
    arg_list = _get_arg_list_with_signature(_signature)
    ret_val = ""

    if isfunction(symbol):
        ret_val = ret_val + f"`def {symbol.__name__}({arg_list})"
        ret_val = _get_return_annotation(_signature, ret_val) + "`"
        
    return ret_val

In [ ]:
def fixture_function(arg_1: str, arg_2) -> None:
    pass


actual = _get_symbol_definition(fixture_function)
expected = """`def fixture_function(arg_1: str, arg_2) -> None
`"""

print(actual)
assert actual == expected

`def fixture_function(arg_1: str, arg_2) -> None
`


In [ ]:
# | export


def _get_sample_markdown_handler_config() -> Markdown:
    md_config = Markdown(extensions=["toc"], extension_configs={})
    return md_config

In [ ]:
sample_md_config = _get_sample_markdown_handler_config()

print(sample_md_config)
assert isinstance(sample_md_config, Markdown)

<markdown.core.Markdown object>


In [ ]:
# | export


def _get_handler(md_config: Markdown) -> PythonHandler:
    handler = get_handler(theme="material")
    handler._update_env(md_config, {'mdx': [], 'mdx_configs': []})
    return handler

In [ ]:
handler = _get_handler(sample_md_config)

print(handler.env.filters["convert_markdown"])
assert handler.env.filters["convert_markdown"]

<bound method BaseRenderer.do_convert_markdown of <mkdocstrings_handlers.python.handler.PythonHandler object>>


In [ ]:
# | export

class ParameterKindMapper(enum.Enum):
    POSITIONAL_ONLY: ParameterKind = ParameterKind.positional_only
    POSITIONAL_OR_KEYWORD: ParameterKind = ParameterKind.positional_or_keyword
    VAR_POSITIONAL: ParameterKind = ParameterKind.var_positional
    KEYWORD_ONLY: ParameterKind = ParameterKind.keyword_only
    VAR_KEYWORD: ParameterKind = ParameterKind.var_keyword

In [ ]:
assert ParameterKindMapper["POSITIONAL_OR_KEYWORD"].value == ParameterKind.positional_or_keyword
assert ParameterKindMapper["KEYWORD_ONLY"].value == ParameterKind.keyword_only

In [ ]:
# | export


def _get_function_parameters(
    symbol: Union[types.FunctionType, Type[Any]]
) -> Parameters:
    sig = signature(symbol)
    params = [param for param in sig.parameters.values()]
    return Parameters(
        *[
            Parameter(
                param.name,
                annotation=Name(source=str(sig.parameters[param.name]).split(": ")[1].split(" =")[0], full=str(sig.parameters[param.name]).split(": ")[1].split(" =")[0])
                if param.annotation is not param.empty
                else None,
                default="{}"
                if param.name == "kwargs"
                else None
                if param.default is param.empty
                else str(param.default),
                kind=ParameterKindMapper[param.kind.name].value,
            )
            for param in params
        ]
    )

In [ ]:
def fixture_function(param1: str, *, param2: int, **kwargs) -> str:
    """Hello I'm a docstring!

    Args:
        param1: Description.
        param2: Description.
        
    Returns:
        A string
    """
    pass

expected = Parameters(
    Parameter("param1", annotation=Name(source="str",full="str"), kind=ParameterKind.positional_or_keyword),
    Parameter("param2", annotation=Name(source="int",full="int"), kind=ParameterKind.keyword_only),
    Parameter("kwargs", kind=ParameterKind.var_keyword, default="{}"),
)

actual = _get_function_parameters(fixture_function)

print(actual)

sig = signature(fixture_function)
for param in sig.parameters:
    print(param)
    assert actual._parameters_dict[param].as_dict() == expected._parameters_dict[param].as_dict()

assert actual._parameters_dict['param1'].as_dict()["kind"] == ParameterKindMapper["POSITIONAL_OR_KEYWORD"].value

<griffe.dataclasses.Parameters object>
param1
param2
kwargs


In [ ]:
# | export


def _get_func_object_for_symbol(symbol):
    function = Function(symbol.__name__, parameters=_get_function_parameters(symbol)) # todo: add returns and decorators
    return function

In [ ]:
actual = _get_func_object_for_symbol(fixture_function)

print(actual)
assert isinstance(actual, Function)

<Function('fixture_function', None, None)>


In [ ]:
# | export


def _generate_markup_for_docstring_section(section, handler: PythonHandler) -> str:
    template = handler.env.get_template(f"docstring/{section.kind.value}.html")
    rendered_html = template.render(section=section, config=handler.default_config)
    return f"{rendered_html}\n"


def _docstring_to_markdown(symbol: Union[types.FunctionType, Type[Any]]) -> str:
    """Converts a docstring to a markdown-formatted string.

    Args:
        docstring: The docstring to convert.

    Returns:
        The markdown-formatted docstring.
    """
#     docstring = Docstring(symbol.__doc__)  # type: ignore
    function = _get_func_object_for_symbol(symbol)
    docstring = Docstring(getdoc(symbol), parent=function)  # type: ignore
    parsed_docstring_sections = parse(docstring, Parser.google)

    md_config = _get_sample_markdown_handler_config()
    handler = _get_handler(md_config)

    ret_val = [
        f"{section.value}\n"
        if section.kind.value == "text"
        else _generate_markup_for_docstring_section(section, handler)
        for section in parsed_docstring_sections
    ]

    return "".join(ret_val)

In [ ]:
def fixture_function(i: str, a: str) -> str:
    """Very cool function

    **f** is a very cool function
    
    Note:
        Execution context is not the same as the one in the notebook because we want examples to work from
        user code. Make sure you compiled the library prior to executing the examples, otherwise you might
        be running them agains an old version of the library.

    Args:
        i: something
        a: something else
        
    Returns:
        This function returns a sample string

    Raises:
        ValueError: If the object has no docstring
        HTTPError: If the object has no docstring
        
    Examples:
        The following snippet prints out greetings for two names:
        ```python
        print("hello {fill in name_1}")
        print("goodbye {fill in name_2}")
        ```
        
    Example:
        The following snippet prints out greetings for two names:
        ```python
        print("hello {fill in name_1}")
        print("goodbye {fill in name_2}")
        ```
        
    Yields:
        This functiron yields something

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)

    """
    pass

actual = _docstring_to_markdown(fixture_function)
print(actual)

griffe: <module>:12: Failed to get 'name: description' pair from ''
griffe: <module>:14: No type or annotation for returned value 1
griffe: <module>:19: Failed to get 'exception: description' pair from ''
griffe: <module>:35: No type or annotation for yielded value 1


Very cool function

**f** is a very cool function

<details class="note">
  <summary>Note</summary>
  <p>Execution context is not the same as the one in the notebook because we want examples to work from
user code. Make sure you compiled the library prior to executing the examples, otherwise you might
be running them agains an old version of the library.</p>
</details>

  <p><strong>Parameters:</strong></p>
  <table>
    <thead>
      <tr>
        <th>Name</th>
        <th>Type</th>
        <th>Description</th>
        <th>Default</th>
      </tr>
    </thead>
    <tbody>
        <tr>
          <td><code>i</code></td>
          <td>
                <code><span data-autorefs-optional="str">str</span></code>
          </td>
          <td><p>something</p></td>
          <td>
              <em>required</em>
          </td>
        </tr>
        <tr>
          <td><code>a</code></td>
          <td>
                <code><span data-autorefs-optional="str">str</span></code>
          </td>
  

In [ ]:
# | export


def get_formatted_docstring_for_symbol(
    symbol: Union[types.FunctionType, Type[Any]]
) -> str:
    """Recursively parses and get formatted docstring of a symbol.

    Args:
        symbol: A Python class or function object to parse the docstring for.

    Returns:
        A formatted docstring of the symbol and its members.

    """

    def traverse(symbol: Union[types.FunctionType, Type[Any]], contents: str) -> str:
        """Recursively traverse the members of a symbol and append their docstrings to the provided contents string.

        Args:
            symbol: A Python class or function object to parse the docstring for.
            contents: The current formatted docstrings.

        Returns:
            The updated formatted docstrings.

        """
        for x, y in getmembers(symbol):
            if not x.startswith("_") or x == "__init__":
                if isfunction(y) and y.__doc__ is not None:
                    contents += f"{_get_symbol_definition(y)}\n\n{_docstring_to_markdown(y)}"
                elif isclass(y) and not x.startswith("__") and y.__doc__ is not None:
                    contents += f"{_get_symbol_definition(y)}\n\n{_docstring_to_markdown(y)}"
                    contents = traverse(y, contents)
        return contents

    contents = (
        f"{_get_symbol_definition(symbol)}\n\n{_docstring_to_markdown(symbol)}"
        if symbol.__doc__ is not None
        else ""
    )
    if isclass(symbol):
        contents = traverse(symbol, contents)
    return contents

In [ ]:
def fixture_function(
    arg_1: str,
    arg_2: Union[List[str], str],
    arg_3: Optional[int],
    arg_4: Optional[str] = None,
) -> str:
    """This is a one line description for the function

    Args:
        arg_1: Argument 1
        arg_2: Argument 2
        arg_3: Argument 3
        arg_4: Argument 4

    Returns:
        The concatinated string
    """
    pass

actual = get_formatted_docstring_for_symbol(fixture_function)
print(actual)


griffe: <module>:9: No type or annotation for returned value 1


`def fixture_function(arg_1: str, arg_2: Union[List[str], str], arg_3: Optional[int], arg_4: Optional[str] = None) -> str
`

This is a one line description for the function

  <p><strong>Parameters:</strong></p>
  <table>
    <thead>
      <tr>
        <th>Name</th>
        <th>Type</th>
        <th>Description</th>
        <th>Default</th>
      </tr>
    </thead>
    <tbody>
        <tr>
          <td><code>arg_1</code></td>
          <td>
                <code><span data-autorefs-optional="str">str</span></code>
          </td>
          <td><p>Argument 1</p></td>
          <td>
              <em>required</em>
          </td>
        </tr>
        <tr>
          <td><code>arg_2</code></td>
          <td>
                <code><span data-autorefs-optional="Union[List[str], str]">Union[List[str], str]</span></code>
          </td>
          <td><p>Argument 2</p></td>
          <td>
              <em>required</em>
          </td>
        </tr>
        <tr>
          <td><code>arg_3</co